# GLO-MIA adapted to use logits instead of label only
Relaxes the constraint that we only get labels, providing logits or probabilities to the model. (Is getting probabilities back a realistic situation?)

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import numpy as np

from torch.nn import functional as F
from torch import nn, optim
from torch.utils.data import TensorDataset
from torch_geometric.loader import DataLoader as GDataLoader
from torch_geometric.datasets import TUDataset
from sklearn.utils.class_weight import compute_class_weight

from ml_util import CustomGATModel, train_model_multi_graph, train_model, load_model, predict, get_auroc_score
from util import onehot_transform, graph_train_test_split, create_perturbed_graphs
from train_models import get_dataset, shadow_target_split

In [9]:
DEVICE = ('cuda:0' if torch.cuda.is_available() else 'cpu')

In [10]:
dataset_name = 'ogbg-molhiv'

In [11]:
dataset = get_dataset(dataset_name)
num_feat = dataset[0].x.shape[1]
num_categories = dataset[0].y.shape[1]

In [12]:
t_dataset_train, t_dataset_test, s_dataset_train, s_dataset_test = shadow_target_split(dataset)

In [18]:
# Fetch target model
t_model = CustomGATModel(num_feat=num_feat, num_classes=num_categories).to(DEVICE)
t_save_path = f'mia-models/t_model_gat_{dataset_name}.pth'

t_model, t_dataset_train, t_dataset_test = load_model(t_model, t_save_path)

In [19]:
# Fetch shadow model
s_model = CustomGATModel(num_feat=num_feat, num_classes=num_categories).to(DEVICE)
s_save_path = f'mia-models/s_model_gat_{dataset_name}.pth'

s_model, s_dataset_train, s_dataset_test = load_model(s_model, s_save_path)

In [20]:
def create_attack_dataset(model, train_dataset, test_dataset, scaler=0.4, n_perturb_per_graph=10, device='cpu'):
    '''Create an attack dataset containing logits that are generated from perturbed graphs'''
    
    model.eval()
    chunksize = 250
    train_loader = GDataLoader(train_dataset, batch_size=chunksize, shuffle=False)
    test_loader = GDataLoader(test_dataset, batch_size=chunksize, shuffle=False)
    all_logits = []
    all_labels = []
    
    with torch.no_grad():
        # Enumerate over each dataset
        for i, loader in enumerate((train_loader, test_loader)):
            label = [1,0] if i == 0 else [0,1]
            for i, gbatch in enumerate(loader):
                gbatch = gbatch.to(device)
                x_p = create_perturbed_graphs(gbatch.x, num=n_perturb_per_graph, scaler=scaler, device=device).to(device)
                pred = torch.stack([model(x_pi, gbatch.edge_index, gbatch.batch).squeeze() for x_pi in x_p])
                all_logits.append((pred.flatten(end_dim=1).sort(dim=1)[0]))
                all_labels.append(torch.Tensor([label] * n_perturb_per_graph * len(gbatch)))        
        
            
    return torch.cat(all_logits), torch.cat(all_labels)

In [1]:
F

NameError: name 'F' is not defined

In [21]:
class GenericAttackModel(nn.Module):
    def __init__(self, num_feat):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(num_feat, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )
        
        
    def forward(self, x):
        return self.layers(x)

In [27]:
s_logits, s_labels = create_attack_dataset(s_model, s_dataset_train, s_dataset_test, scaler=1, device=DEVICE)
t_logits, t_labels = create_attack_dataset(t_model, t_dataset_train, t_dataset_test, scaler=1, device=DEVICE)

In [28]:
att_dataset = TensorDataset(s_logits, s_labels)
target_dataset = TensorDataset(t_logits, t_labels)
lr = 0.002
epochs = 150
batch_size = 32
weight_decay = 1e-3
att_model = GenericAttackModel(num_feat=2).to(DEVICE)
optimizer = optim.Adam(att_model.parameters(), lr=lr, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss()

train_model(att_model, optimizer, att_dataset, loss_fn, epochs, batch_size, val_dataset=target_dataset, device=DEVICE)
att_model.eval()

Learning rate: 0.002
No learning rate scheduling!
Training for 150 epochs, with batch size=32
Using device: cuda:0

-----Epoch 1/150-----
Batch 50/6427 | loss: 0.7057300090789795 (0.068s) | train acc: 0.5594 | train auc: 0.492446
Batch 100/6427 | loss: 0.6821008622646332 (0.066s) | train acc: 0.5813 | train auc: 0.492618
Batch 150/6427 | loss: 0.6890039873123169 (0.066s) | train acc: 0.5842 | train auc: 0.492127
Batch 200/6427 | loss: 0.6775495886802674 (0.066s) | train acc: 0.5887 | train auc: 0.491620
Batch 250/6427 | loss: 0.6854936265945435 (0.066s) | train acc: 0.5889 | train auc: 0.489709
Batch 300/6427 | loss: 0.6816746819019318 (0.066s) | train acc: 0.5889 | train auc: 0.488490
Batch 350/6427 | loss: 0.6710280823707581 (0.066s) | train acc: 0.5929 | train auc: 0.489173
Batch 400/6427 | loss: 0.673120174407959 (0.066s) | train acc: 0.5947 | train auc: 0.489050
Batch 450/6427 | loss: 0.6799440717697144 (0.066s) | train acc: 0.5948 | train auc: 0.490677
Batch 500/6427 | loss: 0.67

KeyboardInterrupt: 

In [42]:
pred_scores_test = predict(att_model, target_dataset, device=DEVICE, logits=True, return_type='pt').cpu()
true_scores_test = target_dataset.tensors[1]
get_auroc_score(pred_scores_test, true_scores_test)

0.5825846698716655